In [ ]:
!pip install -U -qq transformers peft pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 133.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType
import torch

In [ ]:
from transformers import pipeline

ask_llm = pipeline(
    model="Qwen/Qwen3-0.6B",
    device="cuda"
)

prompt = "Who is Naiem Yousefifard?"

answer = ask_llm(prompt)
answer = answer[0]["generated_text"]

print(answer)

Device set to use cuda


Who is Naiem Yousefifard? What does his name mean? What is his current status in the world of sports? What is his nationality? What is his current position in the world of sports? What is his current status in the world of sports? What is his current status in the world of sports? What is his current status in the world of sports? What is his current status in the world of sports? What is his current status in the world of sports? What is his current status in the world of sports?

Okay, let's start by breaking down the question. The user is asking about Naiem Yousefifard's background, current status in sports, nationality, position, and other related details. First, I need to confirm if there's a real person named Naiem Yousefifard. From what I recall, there's a sports figure with the same name, but I'm not sure about the exact details. Let me check my knowledge base.

Naiem Yousefifard is a Nigerian footballer who plays for the Nigeria national team. He has been a key player for Nige

In [ ]:
df = pd.read_json("data.json")

df.head()

,prompt,completion
0,Who is Naiem Yousefifard?,Naiem Yousefifard is an Iranian software engin...
1,When was Naiem Yousefifard born?,"He was born on October 16, 1983, in Tehran, Iran."
2,What are Naiem Yousefifard’s academic degrees?,He holds a Master’s degree in Information Tech...
3,In which competitions did Naiem Yousefifard wi...,He won gold and silver medals at the 2012 Worl...
4,What is Naiem Yousefifard’s field of expertise?,His main expertise is in designing and develop...


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen3-0.6B"
)

In [ ]:
def preprocess_row(row):
  text = row["prompt"] + "\n" + row["completion"]

  tokens = tokenizer(
      text,
      max_length=128,
      truncation=True,
      padding="max_length",
      return_tensors=None
  )

  tokens["labels"] = tokens["input_ids"].copy()

  return tokens

In [ ]:
processed = df.apply(preprocess_row, axis=1)

In [ ]:
train_encoding = {
    "input_ids": [x["input_ids"] for x in processed],
    "attention_mask": [x["attention_mask"] for x in processed],
    "labels": [x["labels"] for x in processed],
}

# train_encoding["input_ids"]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen3-0.6B",
    device_map="cuda",
    dtype=torch.float16
)

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "v_proj"]
)

model = get_peft_model(model, lora_config)

In [ ]:
training_args = TrainingArguments(
    num_train_epochs=10,
    learning_rate=0.001,
    logging_steps=25,
    report_to="none"
)

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __len__(self):
        return len(self.encodings["input_ids"])
    def __getitem__(self, idx):
        return {
            key: torch.tensor(self.encodings[key][idx])
            for key in self.encodings
        }

In [ ]:
train_dataset = CustomDataset(train_encoding)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=20, training_loss=0.08835355043411255, metrics={'train_runtime': 5.3834, 'train_samples_per_second': 18.575, 'train_steps_per_second': 3.715, 'total_flos': 33951213158400.0, 'train_loss': 0.08835355043411255, 'epoch': 10.0})

In [ ]:
trainer.save_model("./my-qwen")
tokenizer.save_pretrained("./my-qwen")

('./my-qwen/tokenizer_config.json',
 './my-qwen/special_tokens_map.json',
 './my-qwen/chat_template.jinja',
 './my-qwen/vocab.json',
 './my-qwen/merges.txt',
 './my-qwen/added_tokens.json',
 './my-qwen/tokenizer.json')

In [ ]:
from transformers import pipeline

ask_llm = pipeline(
    model="./my-qwen",
    tokenizer="./my-qwen",
    device="cuda"
)

prompt = "When was Naiem Yousefifard born?"
# prompt = "Who is Naiem Yousefifard?"

answer = ask_llm(prompt)
answer = answer[0]["generated_text"]

print(answer)

Device set to use cuda


When was Naiem Yousefifard born? He was born on October 16, 1983, in Tehran, Iran.
